In [1]:
import pandas as pd
import cobra
import memote
import os 

In [4]:
#os.chdir('/Users/benjaminreyes/Desktop/Masterarbeit/neutrophil_modeling/neutrophil-modeling/curation')
wd = os.getcwd()
print(wd)

/Users/benjaminreyes/Desktop/Masterarbeit/neutrophil_modeling/Mitocore/Human1_coverage/Statistics


In [5]:
mitocore0= cobra.io.read_sbml_model('/Users/benjaminreyes/Desktop/Masterarbeit/neutrophil_modeling/neutrophil-modeling/curation/Mitocore_Models/Mitocore_Original.xml')
mitocore1= cobra.io.read_sbml_model('/Users/benjaminreyes/Desktop/Masterarbeit/neutrophil_modeling/neutrophil-modeling/curation/Mitocore_Models/Mitocore_Preliminary_Curationl.xml')
mitocore2= cobra.io.read_sbml_model('/Users/benjaminreyes/Desktop/Masterarbeit/neutrophil_modeling/Mitocore/Human1_coverage/Cleaned_Mitocore_changed_metabolites_metanetx.chemical.xml')

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_10fthf_c "10-Formyltetrahydrofolate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_10fthf_c "10-Formyltetrahydrofolate">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_10fthf_m "10-Formyltetrahydrofolate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_10fthf_m "10-Formyltetrahydrofolate">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_13dpg_c "3-Phospho-D-glyceroyl phosphate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_13dpg_c "3-Phospho-D-glyceroyl phosphate">
Use of CHARGE i

In [19]:
def visualization_model_annotations(model, component):
    """"
    model: COBRApy model object (e.g., mitocore)
    component: str, ('reactions', 'metabolites', 'genes')
    """
    counter=0
    for comp in model.__getattribute__(component):
        annot_dict= comp.__dict__.get( '_annotation')
        counter+=1
        print(annot_dict)

def visualization_model_general(model, component):
    """"
    model: COBRApy model object (e.g., mitocore)
    component: str, ('reactions', 'metabolites', 'genes')
    """
    for comp in model.__getattribute__(component):
        comp_complete= comp.__dict__
        print(comp_complete)

def visualization_model_notes(model, component):
    """"
    model: COBRApy model object (e.g., mitocore)
    component: str, ('reactions', 'metabolites', 'genes')
    """
    counter=0
    for comp in model.__getattribute__(component):
        notes_dict= comp.__dict__.get( 'notes')
        counter+=1
        print(notes_dict)

In [48]:
visualization_model_annotations(mitocore2, 'reactions')


{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{'ec-code': ['2.7.1.1', '2.7.1.2', '2.7.1.4', '2.7.1.55', '2.7.1.59', '2.7.1.60'], 'kegg.reaction': 'R01786', 'metanetx.reaction': 'MNXR191953', 'bigg.reaction': 'HEX1', 'Human1': 'MAR04394'}
{'ec-code': ['3.1.3.1', '3.1.3.104', '3.1.3.2', '3.1.3.23', '3.1.3.25', '3.1.3.29', '3.1.3.58', '3.1.3.9'], 'kegg.reaction': 'R01788', 'metanetx.reaction': 'MNXR195425', 'bigg.reaction': 'G6PPer'}
{'ec-code': '5.3.1.9', 'kegg.reaction': 'R02740', 'metanetx.reaction': 'MNXR191975', 'bigg.reaction': 'PGI', 'Human1': 'MAR04381'}
{'ec-code': '2.7.1.11', 'kegg.reaction': 'R04779', 'metanetx.reaction': 'MNXR145952', 'bigg.reaction': 'PFK', 'Human1': 'MAR04379'}
{'ec-code': '3.1.3.11', 'kegg.reaction': 'R04780', 'metanetx.reaction': 'MNXR147046', 'bigg.reaction': 'FBP', 'Human1': 'MAR04

In [78]:

def get_model_ids_for_databases(model, component_type, dict, databases):
    """
    Extract ids from model annotations for various databases across a specified component type.
    
    Parameters:
    - model: COBRApy model object (mitocore)
    - component_type: str, one of 'reactions', 'metabolites', 'genes'
    - dict: dictionary where database ids are located ('annotations', 'notes')
    - databases: list of str, e.g., ['kegg', 'bigg', 'metanetx', 'uniprot'].
    
    Returns:
    - DataFrame with component ID and extracted database ids.
    """
    # Initialize dictionary
    ids_dict = {'model_id': []}

    counters= {db:0 for db in databases}

    for db in databases: 
        ids_dict[db] = []

    # Access model component (e.g., reactions, genes)
    components = getattr(model, component_type)

    for comp in components:
        ids_dict['model_id'].append(comp.id)

        for db in databases:
            value = None
            annotation = getattr(comp, dict, {})

            # Use several naming conventions
            for key in [db, 
                        f"{db}.reaction", 
                        f"{db}.compound", 
                        f"{db}.metabolite", 
                        f"{db}.chemical", 
                        f"{db}.genes",
                        f"{db}.gene",
                        f"{db} id",
                        f"{db.upper()} ID",
                        f"{db.lower()} ID"
                        ]:
                if key in annotation and annotation[key] not in [None, '', 'N/A']:
                    counters[db] += 1
                    value = annotation[key]
                    break

            ids_dict[db].append(value)
            pd.DataFrame(ids_dict)
    
    return counters

def get_model_ids_for_attr(model, component_type, attr_name):
    """
    Extract attribute in the model in the target core-component.
    
    Parameters:
    - model: COBRApy model object (mitocore)
    - component_type: Core component of GEMs, str, one of 'reactions', 'metabolites', 'genes'
    - attr_name: name of the dictionary where target object is located ('notes', '_gpr')

    
    Returns:
    - DataFrame with component id and extracted database ids.
    """
    # Initialize dictionary
    ids_dict = {'model_id': [], attr_name:[]}

    # Access model component (e.g., reactions, genes)
    components = getattr(model, component_type)

    for comp in components:
        ids_dict['model_id'].append(comp.id)
        value = comp.__dict__.get(attr_name, None)
        ids_dict[attr_name].append(value)

    return pd.DataFrame(ids_dict)

def get_model_ids_for_ec_code(model, component_type, dict_name, databases):
    """
    Extract ids from model annotations for various databases across a specified component type.
        
    Parameters:
    - model: COBRApy model object (mitocore)
    - component_type: str, one of 'reactions', 'metabolites', 'genes'
    - dict_name: dictionary where database ids are located ('annotations', 'notes')
    - databases: list of str, e.g., ['kegg', 'ec-code'].
    
    Returns:
    - DataFrame with component ID and extracted database ids.
    """
    # Initialize dictionary
    ids_dict = {'model_id': []}
    counters = {db: 0 for db in databases}

    for db in databases:
        ids_dict[db] = []

    # Access model component (e.g., reactions, genes)
    components = getattr(model, component_type)

    for comp in components:
        ids_dict['model_id'].append(comp.id)

        annotation = getattr(comp, dict_name, {})

        for db in databases:
            value = None
            #search for different nomenclatures
            keys_to_try = [
                db,
                f"{db}.reaction",
                f"{db}.compound",
                f"{db}.metabolite",
                f"{db}.chemical",
                f"{db}.genes",
                f"{db}.gene",
                f"{db} id",
                f"{db.upper()} ID",
                f"{db.lower()} ID"
            ]

            for key in keys_to_try:
                if key in annotation:
                    if annotation[key] not in [None, "", "N/A"]:
                        counters[db] += 1
                        value = annotation[key]
                    break

            ids_dict[db].append(value)

    # Build DataFrame **after loop**
    df = pd.DataFrame(ids_dict)

    return df



# Metabolites

In [8]:
mitocore0_react= get_model_ids_for_databases(mitocore0, 'metabolites', 'notes', ['KEGG', 'RECON2', 'metanetx', 'Human1'])
mitocore1_react= get_model_ids_for_databases(mitocore1, 'metabolites', '_annotation', ['kegg','bigg', 'metanetx','Human1'])
mitocore2_react= get_model_ids_for_databases(mitocore2, 'metabolites', '_annotation', ['kegg','bigg', 'metanetx','Human1'])


print('Mitocore original', mitocore0_react)
print('Mitocore 1st curation', mitocore1_react)
print('Mitocore 2nd curation', mitocore2_react)

Mitocore original {'KEGG': 436, 'RECON2': 436, 'metanetx': 0, 'Human1': 0}
Mitocore 1st curation {'kegg': 436, 'bigg': 325, 'metanetx': 0, 'Human1': 0}
Mitocore 2nd curation {'kegg': 436, 'bigg': 325, 'metanetx': 436, 'Human1': 421}


# Reactions

In [9]:
mitocore0_react= get_model_ids_for_databases(mitocore0, 'reactions', 'notes', ['KEGG', 'Recon2','EC Number', 'Human1'])
mitocore1_react= get_model_ids_for_databases(mitocore1, 'reactions', '_annotation', ['kegg', 'bigg', 'ec-code', 'metanetx', 'Human1'])
mitocore2_react= get_model_ids_for_databases(mitocore2, 'reactions', '_annotation', ['kegg', 'bigg', 'ec-code', 'metanetx', 'Human1'])


print('Mitocore original', mitocore0_react)
print('Mitocore 1st curation', mitocore1_react)
print('Mitocore 2nd curation', mitocore2_react)

Mitocore original {'KEGG': 328, 'Recon2': 432, 'EC Number': 328, 'Human1': 0}
Mitocore 1st curation {'kegg': 324, 'bigg': 0, 'ec-code': 322, 'metanetx': 0, 'Human1': 0}
Mitocore 2nd curation {'kegg': 328, 'bigg': 432, 'ec-code': 323, 'metanetx': 335, 'Human1': 395}


# Genes

In [10]:
mitocore0_react= get_model_ids_for_databases(mitocore0, 'genes', 'notes', ['Uniprot', 'kegg', 'Human1'])
mitocore1_react= get_model_ids_for_databases(mitocore1, 'genes', '_annotation', ['uniprot','kegg', 'Human1'])
mitocore2_react= get_model_ids_for_databases(mitocore2, 'genes', '_annotation', ['uniprot', 'kegg', 'Human1'])


print('Mitocore original', mitocore0_react)
print('Mitocore 1st curation', mitocore1_react)
print('Mitocore 2nd curation', mitocore2_react)

Mitocore original {'Uniprot': 0, 'kegg': 0, 'Human1': 0}
Mitocore 1st curation {'uniprot': 387, 'kegg': 0, 'Human1': 0}
Mitocore 2nd curation {'uniprot': 595, 'kegg': 587, 'Human1': 595}


# --- ec-codes -------

In [22]:
curation_0_ec = []

for reaction in mitocore0.reactions:
    annotations = getattr(reaction, 'notes', {})
    ec_data = annotations.get('EC Number', [])

    # Normalize: ensure ec_data is always a list
    if isinstance(ec_data, str):
        ec_data = [ec_data]

    # iterate over str (ec-code) in a list made of str (ec_data)
    for ec_code in ec_data:
        if isinstance(ec_code, str):
            curation_0_ec.append(ec_code)

print('mitocore original:', len(curation_0_ec), curation_0_ec)

curation_1_ec = []

for reaction in mitocore1.reactions:
    annotations = getattr(reaction, '_annotation', {})
    ec_data = annotations.get('ec-code', [])

    # Normalize: ensure ec_data is always a list
    if isinstance(ec_data, str):
        ec_data = [ec_data]

    # iterate over str (ec-code) in a list made of str (ec_data)
    for ec_code in ec_data:
        if isinstance(ec_code, str):
            curation_1_ec.append(ec_code)

print('mitocore 1 curation:', len(curation_1_ec), curation_1_ec)


curation_2_ec=[]

for reaction in mitocore2.reactions:
    annotations = getattr(reaction, '_annotation', {})
    ec_data = annotations.get('ec-code', [])

    # Normalize: ensure ec_data is always a list
    if isinstance(ec_data, str):
        ec_data = [ec_data]
        
    # iterate over str (ec-code) in a list made of str (ec_data)
    for ec_code in ec_data:
        if isinstance(ec_code, str):
            curation_2_ec.append(ec_code)

print('mitocore curation 2:', len(curation_2_ec), curation_2_ec)

mitocore original: 339 ['2.7.1.1 or 2.7.1.2', '3.1.3.9', '5.3.1.9', '2.7.1.11', '3.1.3.11', '4.1.2.13', '5.3.1.1', '1.2.1.12', '2.7.2.3', '5.4.2.1', '4.2.1.11', '2.7.1.40', '2.7.1.40', '4.1.1.32', '1.1.1.27', '1.1.1.49', '3.1.1.31', '1.1.1.44', '5.3.1.6', '5.1.3.1', '2.2.1.1', '2.2.1.2', '2.2.1.1', '1.2.4.1, 2.3.1.12, 1.8.1.4', '2.3.3.1', '4.2.1.3', '1.1.1.41', '1.1.1.42', '1.2.4.2, 1.8.1.4, 2.3.1.61', '6.2.1.4', '6.2.1.5', '4.2.1.2', '1.1.1.37', '1.6.5.3, 1.6.5.8, 1.6.5.9', '1.3.5.1', '1.10.2.2', '1.9.3.1', '3.6.3.14, 3.6.3.10, 3.6.3.6', '4.1.1.32', '6.4.1.1', '1.1.1.40', '1.1.1.38', '2.6.1.2', '2.3.3.8', '2.7.4.6', '1.6.1.2', '2.7.4.3', '1.1.1.40', '2.6.1.2', '2.7.4.6', '4.2.1.2', '2.7.4.3', '1.1.1.42', '4.2.1.3', '2.3.3.8', '2.6.1.1', '1.1.1.37', 'N/A', 'N/A', '2.6.1.1', '1.1.1.8', '1.1.5.3', '6.2.1.3', '2.3.1.21', '3.6.1.1', 'N/A', '2.3.1.21', '3.6.1.1', '3.1.2.2', '1.3.8.8, 1.5.5.1', '1.3.1.38', '4.2.1.74, 1.1.1.211, 2.3.1.16', '1.3.8.8, 1.5.5.1', '1.3.1.38', '4.2.1.74, 1.1.1.211,

# ---- GPR rules ----


In [ ]:
def get_model_ids_for_attr(model, component_type, attr_name):
    """
    Extract attribute in the model in the target core-component.
    
    Parameters:
    - model: COBRApy model object (mitocore)
    - component_type: Core component of GEMs, str, one of 'reactions', 'metabolites', 'genes'
    - attr_name: name of the dictionary where target object is located ('notes', '_gpr')

    
    Returns:
    - DataFrame with component ID and extracted database ids.
    """
    # Initialize dictionary
    ids_dict = {'model_id': [], attr_name:[]}

    # Access model component (e.g., reactions, genes)
    components = getattr(model, component_type)

    for comp in components:
        ids_dict['model_id'].append(comp.id)
        value = comp.__dict__.get(attr_name, None)
        ids_dict[attr_name].append(value)

    return pd.DataFrame(ids_dict)

In [12]:
gpr_mit_0=get_model_ids_for_attr(mitocore0, 'reactions', '_gpr')
gpr_mit_1=get_model_ids_for_attr(mitocore1, 'reactions', '_gpr')
gpr_mit_2= get_model_ids_for_attr(mitocore2, 'reactions', '_gpr')

print('GPR Mitocore 0:', gpr_mit_0[150:155])
print('GPR Mitocore 1:', gpr_mit_1[150:155])
print('GPR Mitocore 2:', gpr_mit_2[150:155])

GPR Mitocore 0:     model_id                                 _gpr
150    r0638                      ENSG00000116353
151    r0660                      ENSG00000127884
152    r0722                      ENSG00000138796
153    r0724                      ENSG00000167315
154    r1451  ENSG00000117054 and ENSG00000171503
GPR Mitocore 1:     model_id                                 _gpr
150    r0638                      ENSG00000116353
151    r0660                      ENSG00000127884
152    r0722                      ENSG00000138796
153    r0724                      ENSG00000167315
154    r1451  ENSG00000117054 and ENSG00000171503
GPR Mitocore 2:     model_id                                               _gpr
150    r0638                                    ENSG00000116353
151    r0660  ENSG00000127884 or (ENSG00000084754 and ENSG00...
152    r0722  ENSG00000138796 or (ENSG00000060971 and ENSG00...
153    r0724  ENSG00000167315 or (ENSG00000060971 and ENSG00...
154    r1451  (ENSG00000117054 a

In [ ]:
# Split GPR on 'or' 
def gpr_counte(df, model):
    """ function to count the absolute number of GPR 
    splits GPR based on 'or'

    -df: DataFrame with GPRs and model ids 
    -model: str SBML model
    
    retunr: GPR count 
    """
    gpr_absolute= []

    for index, row in df.iterrows():
        gpr = row['_gpr']
        elements_in_gpr = str(gpr).split(' or ')

        for element in elements_in_gpr:
            if element not in [None, 'N/A','']:
                gpr_absolute.append(element)
    print(gpr_absolute)
    return print(f" {model}, has {len(gpr_absolute)} GPRs" )

In [38]:
gpr_0=gpr_counte(gpr_mit_0, 'Mitocore0')
gpr_0=gpr_counte(gpr_mit_1, 'Mitocore1')
gpr_0=gpr_counte(gpr_mit_2, 'Mitocore2')


['ENSG00000156515', 'ENSG00000159399', 'ENSG00000160883', 'ENSG00000106633', 'ENSG00000131482', 'ENSG00000105220', 'ENSG00000152556', 'ENSG00000165140', 'ENSG00000130957', 'ENSG00000149925', 'ENSG00000109107', 'ENSG00000111669', 'ENSG00000111640', 'ENSG00000102144', 'ENSG00000171314', 'ENSG00000164708', 'ENSG00000074800', 'ENSG00000111674', 'ENSG00000108515', 'ENSG00000067225', 'ENSG00000067225', 'ENSG00000124253', 'ENSG00000111716', 'ENSG00000160211', 'ENSG00000130313', 'ENSG00000142657', 'ENSG00000153574', 'ENSG00000235376', 'ENSG00000163931', 'ENSG00000177156', 'ENSG00000163931', 'ENSG00000131828 and ENSG00000168291 and ENSG00000150768 and ENSG00000091140', 'ENSG00000062485', 'ENSG00000100412', 'ENSG00000166411 and ENSG00000101365 and ENSG00000067829', 'ENSG00000182054', 'ENSG00000105953 and ENSG00000119689 and ENSG00000091140', 'ENSG00000163541 and ENSG00000172340', 'ENSG00000136143', 'ENSG00000091483', 'ENSG00000146701', 'ENSG00000115286 and ENSG00000110717 and ENSG00000178127 and

# GPR and ec-code comparison  Preliminary curation, Alignment to Human1

1. Extract model id with gpr from both MitoCore version
2. Extract model id with ec-code from both MitoCore version
3. merge based on model id

In [ ]:
#  curation1 // final curation into a csv file 
# for visualization of the ec code and GPR to see which enzymaatic reactions cannot introduce protein contraint

#  1. Extract model id with gpr from both MitoCore version
gpr_mitocore_1= get_model_ids_for_attr(mitocore1, 'reactions', '_gpr')
gpr_mitocore_2= get_model_ids_for_attr(mitocore2, 'reactions', '_gpr')

#2. Extract model id with ec-code from both MitoCore version
ec_mitocore_1= get_model_ids_for_ec_code(mitocore1, 'reactions','_annotation', ['ec-code'])
ec_mitocore_2= get_model_ids_for_ec_code(mitocore2, 'reactions','_annotation', ['ec-code'])
print(gpr_mitocore_1, ec_mitocore_1)

# 3. merge based on model id
gpr_merged = pd.merge(
    gpr_mitocore_1, 
    gpr_mitocore_2, 
    on='model_id', 
    suffixes=('_preliminary_curation', '_alignment_Human1_')
)

ec_merged = pd.merge(
    ec_mitocore_1, 
    ec_mitocore_2, 
    on='model_id', 
    suffixes=('_preliminary_curation', '_alignment_Human1_')
)


gpr_ec_merged= pd.merge(
    gpr_merged, 
    ec_merged,
    on='model_id', 
    suffixes=('_preliminary_curation', '_alignment_Human1_')
)

print(gpr_ec_merged)
gpr_ec_merged.to_csv('GPR_comparison.csv', index=False)

       model_id _gpr
0      EX_2hb_e     
1       EX_ac_e     
2     EX_acac_e     
3      EX_akg_e     
4    EX_ala_B_e     
..          ...  ...
550         COt     
551         NOt     
552  PCHOLHSTDe     
553        PSt3     
554         PEt     

[555 rows x 2 columns]        model_id ec-code
0      EX_2hb_e    None
1       EX_ac_e    None
2     EX_acac_e    None
3      EX_akg_e    None
4    EX_ala_B_e    None
..          ...     ...
550         COt    None
551         NOt    None
552  PCHOLHSTDe    None
553        PSt3    None
554         PEt    None

[555 rows x 2 columns]
       model_id _gpr_preliminary_curation _gpr_alignment_Human1_  \
0      EX_2hb_e                                                    
1       EX_ac_e                                                    
2     EX_acac_e                                                    
3      EX_akg_e                                                    
4    EX_ala_B_e                                                    
..  

In [ ]:

import re

# extract synthesis, transport, exchange reactions
synthesis = r"^OF_[A-Za-z0-9]+_MitoCore$"
transport = r"^(?!OF_)[A-Za-z0-9]+_MitoCore$" or 
exchange = r"^EX_[A-Za-z0-9_]+$"

df = {'model_id': [], '_gpr_preliminary_curation': [], 'ec-code_preliminary_curation':[], '_gpr_alignment_Human1_': [],'ec-code_alignment_Human1_':[]}

#count all reaction for total of 555 reactio0ns in MitoCore
counter_exchange = 0
counter_transport = 0
counter_core = 0
counter_other = 0

# clasify reaction to return a dataframe just with enzymatic reactions and their features
for index, reaction in gpr_ec_merged.iterrows():
        model_id = reaction['model_id']
        if re.search(synthesis, model_id):
            counter_transport += 1
        elif re.search(transport, model_id):
            counter_core += 1
        elif re.search(exchange, model_id):
            counter_exchange += 1
        else:
            counter_other += 1
            df['model_id'].append(model_id)
            gpr_0 = reaction.get('_gpr_preliminary_curation', None)
            ec_0 = reaction.get('ec-code_preliminary_curation', None)
            ec_1 = reaction.get('ec-code_alignment_Human1_', None)
            gpr_1 = reaction.get('_gpr_alignment_Human1_', None)
            df['_gpr_preliminary_curation'].append(gpr_0)
            df['ec-code_preliminary_curation'].append(ec_0)
            df['_gpr_alignment_Human1_'].append(gpr_1)
            df['ec-code_alignment_Human1_'].append(ec_1)

print(f"synthesis reactions: {counter_transport}")
print(f"transport reactions: {counter_core}")
print(f"Exchange reactions: {counter_exchange}")
print(f"Other reactions: {counter_other}")

df = pd.DataFrame(df)
df.to_csv('enzymatic_react.csv', index=False)
print(df)
    

synthesis reactions: 4
transport reactions: 94
Exchange reactions: 70
Other reactions: 387
       model_id                          _gpr_preliminary_curation  \
0          HEX1  ENSG00000156515 or ENSG00000159399 or ENSG0000...   
1        G6PPer                                    ENSG00000131482   
2           PGI                                    ENSG00000105220   
3           PFK                                    ENSG00000152556   
4           FBP                 ENSG00000165140 or ENSG00000130957   
..          ...                                                ...   
382         COt                                                      
383         NOt                                                      
384  PCHOLHSTDe                                                      
385        PSt3                                                      
386         PEt                                                      

    ec-code_preliminary_curation  \
0             [2.7.1.1, 2.7.1.2]

In [106]:
mitocore1.reactions.get_by_id('r0645')


Reaction identifier,r0645
Name,R_r0645 2-Aminomuconate semialdehyde + NAD+ + H2O --> 2-Aminomuconate + H+ + NADH
Memory address,0x2a5fc1b50
Stoichiometry,am6sa_c + h2o_c + nad_c --> amuco_c + 2.0 h_c + nadh_c 2-Aminomuconate semialdehyde + H2O + NAD --> 2-Aminomuconate + 2.0 H + NADH
GPR,
Lower bound,0.0
Upper bound,1000.0


In [112]:
print('original MitoCore')
print('reactions',len(mitocore0.reactions))
print('metabolites',len(mitocore0.metabolites))
print('genes', len(mitocore0.genes))
print(' MitoCore prelim curation')
print('reactions',len(mitocore1.reactions))
print('metabolites',len(mitocore1.metabolites))
print('genes', len(mitocore1.genes))
print(' MitoCore alignment Human1')
print('reactions',len(mitocore2.reactions))
print('metabolites',len(mitocore2.metabolites))
print('genes', len(mitocore2.genes))



original MitoCore
reactions 555
metabolites 441
genes 391
 MitoCore prelim curation
reactions 555
metabolites 441
genes 387
 MitoCore alignment Human1
reactions 555
metabolites 441
genes 595


# Classify model reactions into exchange, transport and enzymatic reactions 

transport and ecchange reactions do not have ec-codes

In [206]:
exchange = []
transport=[]
enzymatic=[]


def is_transport_reaction(reaction):
    """collect reactions that are transport steps based on the number of compartments"""

    compartments = {met.id.split("_")[-1] for met in reaction.metabolites}
    return len(compartments) > 1

#check exchange reactions (import export, include boundary reactions)
for reaction in mitocore1.reactions:
    if reaction in mitocore1.exchanges:
        exchange.append(reaction)

#check transport (between compartment)
    elif is_transport_reaction(reaction):
        transport.append(reaction)
        
#check enzymatic reactions
    else: 
        enzymatic.append(reaction)

print('exchange',len(exchange))
print('transport',len(transport))
print('enzymatic',len(enzymatic))



Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the mos

exchange 70
transport 162
enzymatic 323


In [208]:
#check reactions with ec-code in lists
gpr=[]
ec_code=[]
gpr_and_ec_code=[]
none=[]

for reaction in transport:
    gpr_val = reaction.gene_reaction_rule if hasattr(reaction, 'gene_reaction_rule') else reaction.gpr
    id = reaction.id
    annotation = reaction.__dict__.get('_annotation', {})
    ec_val = annotation.get('ec-code')
    if gpr_val and ec_val:
        gpr_and_ec_code.append(reaction)
    elif not gpr_val and not ec_val:
        none.append(reaction)
    elif gpr_val and not ec_val:
        gpr.append(reaction)
    elif ec_val and not gpr_val:
        ec_code.append(reaction)
print('reactions with gpr and ec_code', len(gpr_and_ec_code))
print('reactions with no ec_code', len(gpr))
print('reactions with no gpr', len(ec_code))
print('reactions with none', len(none))

reactions with gpr and ec_code 7
reactions with no ec_code 40
reactions with no gpr 0
reactions with none 115


In [199]:
for reaction in ec_code:
    print(reaction.id)

CIV_MitoCore


In [192]:
for reaction in gpr_and_ec_code:
    id = reaction.id
    gpr_val = reaction.gene_reaction_rule if hasattr(reaction, 'gene_reaction_rule') else reaction.gpr
    print(id, 'has GPR:', gpr_val)

for reaction in gpr_and_ec_code:
    id = reaction.id
    annotation = reaction.__dict__.get('_annotation', {})
    ec_val = annotation.get('ec-code')
    print(id, 'has ec-code:', ec_val)

CI_MitoCore has GPR: ENSG00000115286 and ENSG00000110717 and ENSG00000178127 and ENSG00000213619 and ENSG00000158864 and ENSG00000167792 and ENSG00000023228 and ENSG00000198888 and ENSG00000198763 and ENSG00000198840 and ENSG00000198886 and ENSG00000212907 and ENSG00000198786 and ENSG00000198695 and ENSG00000145494 and ENSG00000184752 and ENSG00000164258 and ENSG00000139180 and ENSG00000004779 and ENSG00000131495 and ENSG00000119013 and ENSG00000128609 and ENSG00000184983 and ENSG00000174886 and ENSG00000147123 and ENSG00000168653 and ENSG00000065518 and ENSG00000186010 and ENSG00000099795 and ENSG00000119421 and ENSG00000147684 and ENSG00000140990 and ENSG00000166136 and ENSG00000151366 and ENSG00000090266 and ENSG00000267855 and ENSG00000170906 and ENSG00000189043 and ENSG00000136521 and ENSG00000183648 and ENSG00000109390 and ENSG00000130414 and ENSG00000185633 and ENSG00000160194 and ENSG00000165264
CIII_MitoCore has GPR: (ENSG00000010256 and ENSG00000011295 and ENSG00000127540 and

In [200]:
#this reaction can be fixed
AMCOXO = mitocore1.reactions.get_by_id('CIV_MitoCore')
annotation = AMCOXO.__dict__.get('_annotation', {})
ec_val = annotation.get('ec-code')

print(ec_val)

1.9.3.1


In [201]:
mitocore2.reactions.get_by_id('CIV_MitoCore')

Reaction identifier,CIV_MitoCore
Name,R_CIV_MitoCore cytochrome c oxidase
Memory address,0x2a6a1d950
Stoichiometry,8.0 PMF_m + 4.0 focytC_m + o2_m --> 4.0 PMF_c + 4.0 ficytC_m + 2.0 h2o_m 8.0 Proton Motive Force + 4.0 Ferrocytochrome c + Oxygen --> 4.0 Proton Motive Force + 4.0 Ferricytochrome c + 2.0 H2O
GPR,
Lower bound,0.0
Upper bound,1000.0


list of interesting reactions:
AMCOXO --> ec-code: 1.5.1.-  was deleted